In [1]:
import pandas as pd
import ssl
import json
import urllib
import plotly.graph_objs as go
pd.set_option("max_rows", 100)

In [2]:
covid_url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
ssl._create_default_https_context = ssl._create_unverified_context
covid_json_unformated = urllib.request.urlopen(covid_url).read().decode("utf-8")
covid_json = json.loads(covid_json_unformated)
cdf = pd.DataFrame(covid_json['records'])

In [3]:
cdf

,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days
0,08/02/2021,2021-05,238,8,Afghanistan,AF,AFG,38041757.0,Asia,1.33
1,01/02/2021,2021-04,267,16,Afghanistan,AF,AFG,38041757.0,Asia,2.58
2,25/01/2021,2021-03,713,43,Afghanistan,AF,AFG,38041757.0,Asia,3.34
3,18/01/2021,2021-02,557,45,Afghanistan,AF,AFG,38041757.0,Asia,3.24
4,11/01/2021,2021-01,675,71,Afghanistan,AF,AFG,38041757.0,Asia,4.15
...,...,...,...,...,...,...,...,...,...,...
10642,20/04/2020,2020-16,11,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.11
10643,13/04/2020,2020-15,5,2,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05
10644,06/04/2020,2020-14,2,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05
10645,30/03/2020,2020-13,5,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05


In [4]:
cdf.rename(
    columns={
        "notification_rate_per_100000_population_14-days": "14d_incidence",
        "geoId" : "geo_Id",
        "countriesAndTerritories" : "countries_and_territories",
        "continentExp" : "continent"
    },
    inplace=True
)

In [5]:
cdf['dateRep'] = pd.to_datetime(cdf['dateRep'])
cdf['14d_incidence'] = pd.to_numeric(cdf['14d_incidence'])
cdf.dtypes

dateRep                      datetime64[ns]
year_week                            object
cases_weekly                          int64
deaths_weekly                         int64
countries_and_territories            object
geo_Id                               object
countryterritoryCode                 object
popData2019                         float64
continent                            object
14d_incidence                       float64
dtype: object

In [6]:
cdf['delta_time_since_start_of_recording'] = cdf['dateRep'] - min(cdf['dateRep'])

In [7]:
cdf.describe()

,cases_weekly,deaths_weekly,popData2019,14d_incidence,delta_time_since_start_of_recording
count,1.064700e+04,10647.000000,1.062100e+04,10408.000000,10647
mean,1.000194e+04,218.193200,4.007615e+07,86.880177,226 days 06:38:34.792899408
std,5.912234e+04,1035.493355,1.506333e+08,203.132321,134 days 13:20:23.413722106
min,-3.864000e+03,-875.000000,8.150000e+02,-132.600000,0 days 00:00:00
25%,9.000000e+00,0.000000,1.269670e+06,0.960000,133 days 00:00:00
50%,1.990000e+02,2.000000,7.044639e+06,8.750000,213 days 00:00:00
75%,2.697000e+03,43.000000,2.851583e+07,74.600000,315 days 00:00:00
max,1.782792e+06,23518.000000,1.433784e+09,4343.440000,665 days 00:00:00


In [8]:
# remove negative values
for variable in ['cases_weekly', '14d_incidence', 'deaths_weekly']:
    mask1 = cdf[variable] < 0
    for index in cdf[mask1].index[:]:
        cdf.loc[index, variable] = float('NaN')

In [9]:
copy_cdf = pd.DataFrame.copy(cdf)
#copy_cdf['diff_14d_incidence'] = cdf['14d_incidence']
grp = cdf.groupby(['countries_and_territories'])
for country in cdf['countries_and_territories'].unique():
    grp_country = grp.get_group(country).sort_values('delta_time_since_start_of_recording')
    for pos, incidence in enumerate(grp_country['14d_incidence']):
        if pos > 0:
            copy_cdf.loc[grp_country.index[0], 'diff_14d_incidence'] = (grp_country['14d_incidence'][pos] - grp_country['14d_incidence'][pos-1])/float((grp_country['delta_time_since_start_of_recording'][pos] - grp_country['delta_time_since_start_of_recording'][pos-1]).days)
        else:
            copy_cdf.loc[grp_country.index[0], 'diff_14d_incidence'] = (grp_country['14d_incidence'][pos])
    break
#copy_cdf.groupby(['countries_and_territories'])
#copy_grp_country = copy_cdf.groupby(['countries_and_territories']).get_group(country).sort_values('delta_time_since_start_of_recording')
copy_grp_country
#grp_country

NameError: name 'copy_grp_country' is not defined

In [ ]:
indexed_cdf = cdf.set_index(['continent', 'countries_and_territories', 'delta_time_since_start_of_recording']).sort_values(['continent', 'countries_and_territories', 'delta_time_since_start_of_recording'])

d_14d_incidence = []
for countries in indexed_cdf.index.get_level_values('countries_and_territories').unique:
    for pos, incidence in enumerate(indexed_cdf['14d_incidence']):
        if pos > 0:
            d_14d_incidence.append((indexed_cdf['14d_incidence'][pos] - indexed_cdf['14d_incidence'][pos-1])/float((indexed_cdf.index.get_level_values('delta_time_since_start_of_recording')[pos] - indexed_cdf.index.get_level_values('delta_time_since_start_of_recording')[pos-1]).days))
        else:
            d_14d_incidence.append(indexed_cdf['14d_incidence'][pos])
print(d_14d_incidence)
#for countries in sorted_cdf['countries_and_territories']:
#    for pos, incidence in enumerate(sorted_cdf['14d_incidence']):
#        if pos == 0:
#            sorted_cdf['diff_14d_incidence'][pos] = sorted_cdf['14d_incidence'][pos]
#        else:
#            sorted_cdf['diff_14d_incidence'][pos] = (sorted_cdf['14d_incidence'][pos] - sorted_cdf['14d_incidence'][pos-1])/(sorted_cdf['deltaTime_since_start_of_recording'][pos] - sorted_cdf['deltaTime_since_start_of_recording'][pos-1])

In [ ]:
copy_grp_country